In [1]:
import pygame
import sys
import numpy as np
import random
import pandas as pd
import time
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
BATCH_SIZE=32


pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
width=10
height=10
pixel_size=(50,50)
display_size=(pixel_size[0]*width,pixel_size[1]*height)
record=[]
game_num=1
p=0
step=0

color_dict={
    "wall":(177,177,177),
    "target":(0,177,0),
    "normal":(255,255,255),
    "start":(177,0,0)
}

map_dict={
    "target":(7,2),
    "start":(2,height-1),
    "tank1":(7,4),
    "tank2":(5,2),
    "tank3":(random.randint(3,width-3),random.randint(2,height-3)),

   }

wall_location=[]
for w in range(width):
    wall_location.append((w,0))
    wall_location.append((w,height-1))
    
for h in range(height):    
    wall_location.append((0,h))
    wall_location.append((width-1,h))
    
wall_location.remove(map_dict.get("start"))
wall_location.append((map_dict.get("start")[0],map_dict.get("start")[1]+1))
map_dict['wall']=wall_location

df100=pd.DataFrame(columns=['s','a','s_','r'])
df1=pd.DataFrame(columns=['s','a','s_','r'])
df0=pd.DataFrame(columns=['s','a','s_','r'])

epsilon=0
reward=[-0.02]
gamma=0.9
a_dict={
    0:'up',
    1:'down',
    2:'left',
    3:'right'
}

s_index={}
for i in range(width-2):
    for j in range(height-2):
        s_index.update({(i+1,j+1):(i+(j*(width-2)))})

s_index.update({map_dict.get("start"):(width-2)*(height-2)})   


In [3]:
def draw_map():
    for i in range(width*height):
        w=i-width*(i//width)
        h=i//width
        
        if (w,h)==map_dict.get("target"):
            color=color_dict.get("target")
        elif (w,h)==map_dict.get("start"):
            color=color_dict.get("start")
        elif (w,h) in map_dict.get("wall"):
            color=color_dict.get("wall")
        else:
            color=color_dict.get("normal")
        
        pygame.draw.rect(game_map,color,(w*50,h*50,pixel_size[0],pixel_size[1]))

        
class tank(pygame.sprite.Sprite):
    def __init__(self,filename,location):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        self.image = pygame.transform.scale(self.image,(50,50))

        self.rect = self.image.get_rect()
        self.rect.topleft=(location[0]*pixel_size[0],location[1]*pixel_size[1])

        
def crash_wall():
    if (person_site[0],person_site[1]) in map_dict.get("wall"):
        return True
    else:
        return False
    
    
def crash_detect(sprite1,sprite2):
    global game_num
    global game_state
    result=pygame.sprite.collide_mask(sprite1,sprite2)
    
    if result:
        if sprite2 in tank1_group:
            print("这是第{}局游戏，我死了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(0)
            reward.append(-1)

        elif sprite2 in princess_group:
            print("这是第{}局游戏，我赢了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(1)
            reward.append(1)

        game_num+=1
        game_state=1
        
        
def move(direction):
    global person_site
    global step
    if direction=='up':
        person_site[1]-=1
        a= crash_wall()
        if a==True:
            person_site[1]+=1
        
    if direction=='down':
        person_site[1]+=1
        a= crash_wall()
        if a==True:
            person_site[1]-=1
    if direction=='left':
        person_site[0]-=1
        a= crash_wall()
        if a==True:
            person_site[0]+=1
    if direction=='right':
        person_site[0]+=1
        a= crash_wall()
        if a==True:
            person_site[0]-=1
        
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    crash_detect(person,tank1)       
    crash_detect(person,tank2)   
    crash_detect(person,tank3)       

    crash_detect(person,princess)         
    step+=1

In [8]:
def pick_a(state):    
    if random.random()>epsilon:
        action=random.randint(0,3)
    else:
        x=model1.predict([state])
        action= x.argmax()
    return action


def one_turn():
    s1=int(person_site[0])
    s2=int(person_site[1])
    a=pick_a([s1,s2])
    move(a_dict.get(a)) 
    s1_=int(person_site[0])
    s2_=int(person_site[1])
    r=reward[-1]
    trace_collect(s1,s2,a,s1_,s2_,r)
    
    if game_num>50 and step%200==0:
        fit(10)
        
    if game_state==1:           
        reset()

        
def trace_collect(s1,s2,a,s1_,s2_,r):   
    global df100
    global df1
    global df0
    new_trace={
        's1':s1,
        's2':s2,
        'a':a,
        's1_':s1_,
        's2_':s2_,
        'r':r
    }
    if r==1:
        df100=df100.append(new_trace,ignore_index=True)
        df100=df100.iloc[-100:]
        #df100.to_csv('trace100.csv')
    if r==-1:
        df1=df1.append(new_trace,ignore_index=True)
        df1=df1.iloc[-1000:]
        #df1.to_csv('trace1.csv')
    else:
        df0=df0.append(new_trace,ignore_index=True)
        df0=df0.iloc[-4000:]
        #df0.to_csv('trace0.csv')
    
def preprocessing(batch_size):
    #df100=pd.read_csv('trace100.csv')
    #df1=pd.read_csv('trace1.csv')
    #df0=pd.read_csv('trace0.csv')
    ds100=df100.sample(batch_size,replace = True)
    ds1=df1.sample(batch_size//2,replace = True)
    ds0=df0.sample(batch_size//2,replace = True)
    train_ds=pd.concat((ds100,ds1,ds0))
    train_ds=train_ds.sample(batch_size)
    return train_ds
    
    

In [9]:
def train_step(train_ds):
    global p
    s = np.int32(np.stack((train_ds['s1'],train_ds['s2']),1))
    a = np.array(train_ds['a'])
    s_ = np.int32(np.stack((train_ds['s1_'],train_ds['s2_']),1))
    r = np.array(train_ds['r'])

    
    c1=np.zeros((len(train_ds),4))   #A方向外全为0的矩阵
    for i in range(len(a)):
        c1[i][int(a[i])]=1
    
    c2=np.ones((len(train_ds),4))    #A方向外全为1的矩阵
    for i in range(len(a)):
        c2[i][int(a[i])]=0
        
    d=np.ones((len(r)))
    for i in range(len(r)):
        if r[i]==1:
            d[i]=0
        elif r[i]==-1:
            d[i]=0
        else:
            pass
    
    qloss=model1(s) #取qt以外的值
    qt=model2(s_)   #取max的值
    qtarget=tf.reduce_max(qt,axis=1)*gamma*d+r
    qtloss=tf.transpose([qtarget,qtarget,qtarget,qtarget])*c1+qloss*c2
    

   
    with tf.GradientTape(persistent=True) as tape:
        #loss函数
        q=model1(s)                     
        loss=tf.reduce_sum(tf.square(qtloss-q))
        p=float(loss)
    
    gradient=tape.gradient(loss,model1.trainable_variables)
    opt.apply_gradients(zip(gradient,model1.trainable_variables))
    del tape

    

In [10]:
model1=tf.keras.Sequential([
    tf.keras.layers.Dense(4096,input_shape=(BATCH_SIZE,2),activation='relu'),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(4)       #a,qmax=r+gamma*q_max
   ]
)

model2=tf.keras.Sequential([
    tf.keras.layers.Dense(4096,input_shape=(BATCH_SIZE,2),activation='relu'),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(4)       #a,qmax=r+gamma*q_max
   ]
)

opt=tf.keras.optimizers.Adam(learning_rate=0.001)

In [12]:
def fit(epochs):
    for epoch in range(epochs):
        train_ds=preprocessing(BATCH_SIZE)
        for i in range(50):
            train_step(train_ds)
        print(str(p)+'.',end='')
        
        if (epoch+1)%10==0:
            print("EPOCH:",epoch+1)

    
def load_weight():
    model1.save_weights('DQN.h5')
    model2.load_weights('DQN.h5')
    
    
def debug():
    for i in range(width-2):
        for j in range(height-2):
            result=model1.predict([[i+1,j+1]])
            print(i+1,j+1)
            print(result)
            
            
def reset():
    global person_site
    global game_state
    person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    game_state=0
    reward.append(-0.02)
    
    if game_num>70 and game_num%20==0:    
        load_weight()
        debug()

In [14]:
pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("AI训练游戏")

screen = pygame.display.set_mode(display_size)
game_map=pygame.surface.Surface(display_size)
draw_map()

tank1_group = pygame.sprite.Group()
tank2_group = pygame.sprite.Group()
princess_group = pygame.sprite.Group()
person_group=pygame.sprite.Group()

tank1 = tank("img/tank.jpg",map_dict.get("tank1"))
tank2 = tank("img/tank.jpg",map_dict.get("tank2"))
tank3 = tank("img/tank.jpg",map_dict.get("tank3"))

tank1_group.add(tank1)
tank1_group.add(tank2)
tank1_group.add(tank3)


princess=tank('img/sidalin.jpeg',map_dict.get("target"))
princess_group.add(princess)

person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
person=tank('img/maintank.jpeg',person_site)
person_group.add(person)

reset()

while True:
    clock.tick(60)

    screen.blit(game_map,(0,0))
    tank1_group.draw(screen)
    princess_group.draw(screen)
    screen.blit(person.image,person.rect)
    tank2_group.draw(screen)
    
    
    one_turn()



    if game_num==50:
        epsilon=0.2
    elif game_num==100:
        epsilon=0.3   
    elif game_num==150:
        epsilon=0.4
    elif game_num==200:
        epsilon=0.5
    elif game_num==250:
        epsilon=0.6
    elif game_num==300:
        epsilon=0.7
    elif game_num==350:
        epsilon=0.8
    elif game_num==400:
        epsilon=0.9
        
    for event in pygame.event.get():
        if event.type ==pygame.QUIT:
            pygame.quit()
            sys.exit()
            

        
#       if event.type ==pygame.KEYDOWN:
#           if event.key == pygame.K_UP:
#               move('up')
#           if event.key == pygame.K_DOWN:
#               move('down')
#           if event.key == pygame.K_LEFT:
#               move('left')
#           if event.key == pygame.K_RIGHT:
#               move('right')
#           if event.key == pygame.K_SPACE:
                
                
     
    pygame.display.update()

这是第1局游戏，我死了
这是第2局游戏，我死了
这是第3局游戏，我死了
这是第4局游戏，我死了
这是第5局游戏，我死了
这是第6局游戏，我死了
这是第7局游戏，我死了
这是第8局游戏，我死了
这是第9局游戏，我死了
这是第10局游戏，我死了
这是第11局游戏，我死了
这是第12局游戏，我死了
这是第13局游戏，我死了
这是第14局游戏，我赢了
这是第15局游戏，我死了
这是第16局游戏，我死了
这是第17局游戏，我赢了
这是第18局游戏，我死了
这是第19局游戏，我死了
这是第20局游戏，我死了
这是第21局游戏，我死了
最后20盘的胜率为0.1
这是第22局游戏，我死了
最后20盘的胜率为0.1
这是第23局游戏，我死了
最后20盘的胜率为0.1
这是第24局游戏，我死了
最后20盘的胜率为0.1
这是第25局游戏，我死了
最后20盘的胜率为0.1
这是第26局游戏，我死了
最后20盘的胜率为0.1
这是第27局游戏，我死了
最后20盘的胜率为0.1
这是第28局游戏，我死了
最后20盘的胜率为0.1
这是第29局游戏，我死了
最后20盘的胜率为0.1
这是第30局游戏，我死了
最后20盘的胜率为0.1
这是第31局游戏，我死了
最后20盘的胜率为0.1
这是第32局游戏，我死了
最后20盘的胜率为0.1
这是第33局游戏，我死了
最后20盘的胜率为0.1
这是第34局游戏，我死了
最后20盘的胜率为0.1
这是第35局游戏，我死了
最后20盘的胜率为0.05
这是第36局游戏，我死了
最后20盘的胜率为0.05
这是第37局游戏，我死了
最后20盘的胜率为0.05
这是第38局游戏，我死了
最后20盘的胜率为0.0
这是第39局游戏，我死了
最后20盘的胜率为0.0
这是第40局游戏，我死了
最后20盘的胜率为0.0
这是第41局游戏，我死了
最后20盘的胜率为0.0
这是第42局游戏，我死了
最后20盘的胜率为0.0
这是第43局游戏，我死了
最后20盘的胜率为0.0
这是第44局游戏，我死了
最后20盘的胜率为0.0
这是第45局游戏，我死了
最后20盘的胜率为0.0
这是第46局游戏，我死了
最后20盘的胜率为0.0
这是第47局游戏，我死了
最后20盘的胜率为0.0
这是第48局游戏，我死了
最后20盘的胜率为0.0
这是第49局游戏，我死了
最后20盘

5 8
[[0.32405287 0.3571371  0.29397187 0.29952222]]
6 1
[[ 0.4718917  -1.4875715   1.1515639   0.84601486]]
6 2
[[ 0.87651867 -0.69678533 -0.48647907  1.0009836 ]]
6 3
[[ 0.27708027 -0.50972724 -0.6394199   0.1386062 ]]
6 4
[[-1.208268  -0.8926442 -0.6771288 -0.8274734]]
6 5
[[-1.1252874  -0.73555946 -0.9835806  -0.5337806 ]]
6 6
[[-0.18379048  0.05588286 -0.06456562  0.07152796]]
6 7
[[0.21912515 0.32198057 0.32082528 0.28628707]]
6 8
[[0.3073733  0.3750963  0.42002383 0.30857188]]
7 1
[[ 0.5045506 -1.9785109  1.7032042  1.0261805]]
7 2
[[ 0.8873381  -1.1479087   0.18623315  1.2118933 ]]
7 3
[[ 0.9996421  -0.58821493 -0.91927975  1.0635473 ]]
7 4
[[ 0.01027374 -0.41032082 -0.46435645 -0.05656845]]
7 5
[[-1.000271   -0.44767153 -0.61849296 -0.4491229 ]]
7 6
[[0.16844288 0.19153321 0.1385007  0.34713763]]
7 7
[[0.3900505  0.33506003 0.38429475 0.3969636 ]]
7 8
[[0.5080657  0.3964546  0.5089183  0.45938563]]
8 1
[[ 0.58718026 -2.4819782   2.2151124   1.2503586 ]]
8 2
[[ 0.7945685 -1.6756

7 2
[[ 1.0432959 -1.2379172  0.6333072  1.4502202]]
7 3
[[ 0.99869794 -0.84911203 -0.17152327  0.9458713 ]]
7 4
[[-0.16324604 -0.39347565 -0.13223132 -0.12186924]]
7 5
[[-1.0572915  -0.00863021 -0.03251209 -0.7260498 ]]
7 6
[[-0.3650345   0.12769441 -0.23818749 -0.19048774]]
7 7
[[0.23003976 0.3910228  0.19869362 0.36312243]]
7 8
[[0.269034   0.46991757 0.38057864 0.4966775 ]]
8 1
[[ 1.3297906 -2.0214152  1.5384353  2.4527051]]
8 2
[[ 1.2593718  -1.5834472   0.99976885  1.925817  ]]
8 3
[[ 1.234987   -1.1705072   0.18478787  1.4008555 ]]
8 4
[[ 0.8072118  -0.7427844  -0.19272295  0.79244107]]
8 5
[[-0.4742344  -0.13771823  0.10125817 -0.0662591 ]]
8 6
[[-0.3289953   0.31418592  0.28669232  0.11757783]]
8 7
[[0.19407138 0.42477047 0.3186697  0.40107092]]
8 8
[[0.3049212  0.52803934 0.40242046 0.56064826]]
这是第100局游戏，我死了
最后20盘的胜率为0.0
这是第101局游戏，我死了
最后20盘的胜率为0.0
3.564738653949462e-05.0.00028807666967622936.0.0012098749866709113.0.0002354854514123872.3.5729339288081974e-05.0.0004137563810218

0.00024464158923365176.0.0008821288356557488.0.008604444563388824.0.0006924283225089312.0.0004985264968127012.0.0010910441633313894.0.00022586988052353263.0.0016986941918730736.0.0008767362451180816.0.00025267808814533055.EPOCH: 10
这是第117局游戏，我死了
最后20盘的胜率为0.15
3.63640756404493e-05.0.001468862290494144.0.0017159453127533197.0.0005162634188309312.0.00033087562769651413.0.0005534752272069454.0.0002726476523093879.3.898413706338033e-05.0.0004051618161611259.0.0007720603025518358.EPOCH: 10
0.011565589345991611.0.0006892010569572449.0.0009000037098303437.0.0028855339623987675.0.0014339950866997242.0.0003683138347696513.0.0023130530025810003.0.0007361542666330934.0.005030449479818344.0.00434779142960906.EPOCH: 10
这是第118局游戏，我死了
最后20盘的胜率为0.15
这是第119局游戏，我死了
最后20盘的胜率为0.15
1 1
[[0.09434748 0.195568   0.16330273 0.15434852]]
1 2
[[0.04650113 0.20509523 0.19870782 0.09157787]]
1 3
[[0.04182306 0.223283   0.24617995 0.06509282]]
1 4
[[0.05602941 0.23040712 0.2830261  0.04159036]]
1 5
[[0.09288452 0.20

0.0010726326145231724.0.0012015084503218532.0.0002236224536318332.0.0005900414544157684.7.753349927952513e-05.2.2767357222619466e-05.0.004341450519859791.0.000290597730781883.0.00034341742866672575.2.1268369891913608e-05.EPOCH: 10
这是第154局游戏，我死了
最后20盘的胜率为0.15
0.0001672750513534993.0.0008436198113486171.6.255646439967677e-05.0.0002647065557539463.0.0007177068619057536.0.00034271314507350326.0.0004698332049883902.0.0014422982931137085.0.0005437115905806422.1.5502662790822797e-05.EPOCH: 10
9.719482477521524e-05.0.000146682039485313.0.00011046582949347794.0.00025568457203917205.0.00028611739980988204.4.605500362231396e-05.0.00014652042591478676.0.0002857492072507739.0.0005229452508501709.0.0005950030172243714.EPOCH: 10
这是第155局游戏，我死了
最后20盘的胜率为0.15
这是第156局游戏，我赢了
最后20盘的胜率为0.15
0.00023626464826520532.0.0008509205072186887.0.00015289784641936421.0.00031764397863298655.7.159856613725424e-05.0.0008490970940329134.0.00015517201973125339.2.0794832380488515e-05.0.00040330091724172235.6.49740977678448

...

5 8
[[0.23764159 0.25585356 0.24201176 0.26207364]]
6 1
[[1.1776377 0.6542001 1.0991076 1.3541577]]
6 2
[[ 0.9360275  -0.9240183  -1.0123109   0.99908435]]
6 3
[[-0.6093545  -1.150493   -1.1993982  -0.35175818]]
6 4
[[-1.0555055  -1.0708724  -0.99832374 -1.0313078 ]]
6 5
[[-1.0055704  -0.25527743 -0.9115563  -0.6514162 ]]
6 6
[[-0.65468085  0.23395759  0.2185503   0.24902187]]
6 7
[[0.23986022 0.23618457 0.24150866 0.24887516]]
6 8
[[0.23675753 0.26754755 0.242744   0.2700946 ]]
7 1
[[1.3312186 0.99912   1.3461531 1.4744141]]
7 2
[[1.3101492 0.5721535 0.6735502 1.1124895]]
7 3
[[ 0.99944454 -0.9894648  -1.0807911   0.6454301 ]]
7 4
[[-0.443236   -1.1362916  -1.07502    -0.67200726]]
7 5
[[-1.0002623   0.08885241 -0.5372973   0.09245893]]
7 6
[[-0.12629472  0.23311898  0.254398    0.22423075]]
7 7
[[0.23908645 0.2478995  0.24235359 0.2569552 ]]
7 8
[[0.23600529 0.27923095 0.24360603 0.2781607 ]]
8 1
[[1.4994754 1.4694788 1.3672777 1.4911683]]
8 2
[[1.4912087 1.2131568 0.9990675 1.353658

6 3
[[-0.6963804 -1.0902306 -1.1165309 -0.6143311]]
6 4
[[-0.9986421 -1.0683917 -1.0768874 -0.9997828]]
6 5
[[-0.999984   -0.90292215 -1.0021529  -1.0138396 ]]
6 6
[[-0.97037697 -0.16690703 -0.86765337 -0.55039155]]
6 7
[[-0.7162032   0.19970477 -0.07064095  0.11942437]]
6 8
[[0.05079344 0.23187602 0.35142076 0.2286213 ]]
7 1
[[1.2873458 0.998648  1.2683631 1.3356023]]
7 2
[[1.2791107  0.64501864 0.5724543  1.0765449 ]]
7 3
[[ 0.99733245 -0.9973949  -0.86626816  0.75278944]]
7 4
[[-0.18260962 -1.1134107  -1.066426   -0.2525822 ]]
7 5
[[-0.99915063 -1.00783    -1.0495683  -1.0049343 ]]
7 6
[[-0.8368877   0.15704954 -0.7461274  -0.2783414 ]]
7 7
[[-0.4762119   0.24403508  0.29704243  0.2522127 ]]
7 8
[[0.25708532 0.24363485 0.359121   0.26874906]]
8 1
[[1.3278843 1.3289572 1.3603956 1.32449  ]]
8 2
[[1.329405  1.200893  0.9991415 1.1561198]]
8 3
[[ 1.2858294   0.7070968  -0.11415845  0.7567332 ]]
8 4
[[ 1.0616183  -0.8390164  -0.99318993  0.66020614]]
8 5
[[ 0.33894807 -0.5019119  -0.904

3.884760008077137e-05.0.0007680822745896876.0.00012824294390156865.0.00027836693334393203.2.6804777007782832e-05.0.000165293226018548.0.00012496561976149678.2.167853199352976e-05.0.00026073891785927117.0.00013103093078825623.EPOCH: 10
这是第430局游戏，我赢了
最后20盘的胜率为0.55
这是第431局游戏，我赢了
最后20盘的胜率为0.55
这是第432局游戏，我赢了
最后20盘的胜率为0.55
这是第433局游戏，我赢了
最后20盘的胜率为0.55
0.0002287963725393638.2.0874223991995677e-05.0.001135586528107524.2.3909276933409274e-05.9.686697922006715e-06.3.4128446714021266e-05.1.9161070667905733e-05.0.0002769879065454006.2.769919046841096e-05.0.0010485081002116203.EPOCH: 10
这是第434局游戏，我赢了
最后20盘的胜率为0.55
6.861775909783319e-05.0.00010890587145695463.0.0008370594587177038.5.282629354042001e-05.2.3540287656942382e-05.5.1083719881717116e-05.6.217053851287346e-06.6.680017122562276e-06.0.0001921308576129377.0.0003375934611540288.EPOCH: 10
这是第435局游戏，我赢了
最后20盘的胜率为0.6
这是第436局游戏，我死了
最后20盘的胜率为0.6
3.930009188479744e-05.0.0005602521123364568.0.0003145897062495351.3.0022878490854055e-05.0.00023820922069

SystemExit: 

d:\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
